In [1]:
import numpy as np
from numpy import cov, trace, iscomplexobj
from scipy.linalg import sqrtm
from keras.applications.inception_v3 import InceptionV3
from keras.applications.inception_v3 import preprocess_input
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split,cross_val_score
from sklearn.metrics import precision_recall_fscore_support
import tensorflow as tf
from tensorflow.keras.optimizers import Adam

# Using Frechet inception distance on 3d data

In [2]:
# calculate frechet inception distance
def calculate_fid(model, images1, images2):
	# calculate activations
	act1 = model.predict(images1)
	act2 = model.predict(images2)
	# calculate mean and covariance statistics
	mu1, sigma1 = act1.mean(axis=0), cov(act1, rowvar=False)
	mu2, sigma2 = act2.mean(axis=0), cov(act2, rowvar=False)
	# calculate sum squared difference between means
	ssdiff = np.sum((mu1 - mu2)**2.0)
	# calculate sqrt of product between cov
	covmean = sqrtm(sigma1.dot(sigma2))
	# check and correct imaginary numbers from sqrt
	if iscomplexobj(covmean):
		covmean = covmean.real
	# calculate score
	fid = ssdiff + trace(sigma1 + sigma2 - 2.0 * covmean)
	return fid

In [3]:
# prepare the inception v3 model
model = InceptionV3(include_top=False, pooling='avg', input_shape=(80,640,3))

## Generated Using Daniel 3d Input

In [4]:
original_3d_d = np.load("assets/3d_complete_data_Daniel.npz", allow_pickle=True)['arr_0']
generated_3d_d = np.load("generated_data/3d_daniel_gen.npz", allow_pickle=True)['arr_0']
random = np.random.random(generated_3d_d.shape)
random = preprocess_input(random)

In [5]:
# pre-process images
images1 = preprocess_input(original_3d_d)
images2 = preprocess_input(generated_3d_d)
# fid between images1 and images1
fid = calculate_fid(model, images1, images1)
print('FID (same): %.3f' % fid)
# fid between images1 and images2
fid = calculate_fid(model, images1, images2)
print('FID (ours): %.3f' % fid)
fid = calculate_fid(model, images1, random)
print('FID (baseline): %.3f' % fid)

125/125 [==============================] - 83s 660ms/step
FID (same): 0.000
32/32 [==============================] - 19s 555ms/step
FID (ours): 142.336
32/32 [==============================] - 18s 554ms/step
FID (baseline): 173.238


## Generated Using our 3d Input

In [6]:
model = InceptionV3(include_top=False, pooling='avg', input_shape=(80,640,3))

In [7]:
original_3d = np.load("assets/3d_complete_data.npz", allow_pickle=True)['arr_0']
generated_3d = np.load("generated_data/3d_gen.npz", allow_pickle=True)['arr_0']

In [8]:
# pre-process images
images1 = preprocess_input(original_3d)
images2 = preprocess_input(generated_3d)

In [9]:
# fid between images1 and images1
fid = calculate_fid(model, images1, images1)
print('FID (same): %.3f' % fid)

125/125 [==============================] - 72s 575ms/step
FID (same): 0.000


In [10]:
# fid between images1 and images2
fid = calculate_fid(model, images1, images2)
print('FID (ours): %.3f' % fid)

32/32 [==============================] - 22s 611ms/step
FID (ours): 291.836


In [11]:
fid = calculate_fid(model, images1, random)
print('FID (baseline): %.3f' % fid)

32/32 [==============================] - 19s 605ms/step
FID (baseline): 308.623


# Using Other Classifier as Metrics

## SVM (linear, poly and rbf classifer)

In [2]:
def makeSVM():
	# param_grid={'C':[0.1,1,10,100],'gamma':[0.0001,0.001,0.1,1],'kernel':['linear','rbf','poly']}
	model=SVC(kernel='linear',probability=True)
	# model=GridSearchCV(svc,param_grid)
	return model

In [4]:
total_labels = np.concatenate((np.zeros(4000), np.ones(1000)))

### Generated Using Daniel 3d (x,y,z) Input

In [5]:
original_3d_d = np.load("assets/3d_complete_data_Daniel.npz", allow_pickle=True)['arr_0']
generated_3d_d = np.load("generated_data/3d_daniel_gen.npz", allow_pickle=True)['arr_0']
total_images_3d_d = np.vstack((original_3d_d, generated_3d_d))
X_train_3d_d, X_test_3d_d, y_train_3d_d, y_test_3d_d = train_test_split(total_images_3d_d, total_labels, test_size=0.33, random_state=42)

In [6]:
%%time
model = makeSVM()
model.fit(X_train_3d_d.reshape(X_train_3d_d.shape[0],640*80*3), y_train_3d_d)

CPU times: total: 22min 19s
Wall time: 11min 46s


SVC(kernel='linear', probability=True)

In [10]:
pred_y = model.predict(X_test_3d_d.reshape(X_test_3d_d.shape[0],640*80*3))
precision_recall_fscore_support(y_test_3d_d, pred_y, average='weighted')

(1.0, 1.0, 1.0, None)

In [ ]:
# %%time
# scores = cross_val_score(model, total_images_3d_d.reshape(total_images_3d_d.shape[0],640*80*3), total_labels, cv=4)
# print(scores)
# print("%0.2f accuracy with a standard deviation of %0.2f" % (scores.mean(), scores.std()))

### Generated Using Our 3d (x,y,z) Input

In [11]:
original_3d = np.load("assets/3d_complete_data.npz", allow_pickle=True)['arr_0']
generated_3d = np.load("generated_data/3d_gen.npz", allow_pickle=True)['arr_0']
total_images_3d = np.vstack((original_3d, generated_3d))
X_train_3d, X_test_3d, y_train_3d, y_test_3d = train_test_split(total_images_3d, total_labels, test_size=0.33, random_state=42)

In [12]:
%%time
model = makeSVM()
model.fit(X_train_3d.reshape(X_train_3d.shape[0],640*80*3), y_train_3d)

CPU times: total: 3min 4s
Wall time: 1min 41s


SVC(kernel='linear', probability=True)

In [13]:
pred_y = model.predict(X_test_3d.reshape(X_test_3d.shape[0],640*80*3))
precision_recall_fscore_support(y_test_3d, pred_y, average='weighted')

(1.0, 1.0, 1.0, None)

In [ ]:
# %%time
# scores = cross_val_score(model, total_images_3d.reshape(total_images_3d.shape[0],640*80*3), total_labels, cv=4)
# print(scores)
# print("%0.2f accuracy with a standard deviation of %0.2f" % (scores.mean(), scores.std()))

### Generated Using Our 2d (d,z) Input

In [5]:
original_2d = np.load("assets/2d_complete_data.npz", allow_pickle=True)['arr_0']
generated_2d = np.load("generated_data/2d_gen.npz", allow_pickle=True)['arr_0']
total_images_2d = np.vstack((original_2d, generated_2d))
X_train_2d, X_test_2d, y_train_2d, y_test_2d = train_test_split(total_images_2d, total_labels, test_size=0.33, random_state=42)

In [6]:
%%time
model = makeSVM()
model.fit(X_train_2d.reshape(X_train_2d.shape[0],640*80*2), y_train_2d)

CPU times: total: 1min 46s
Wall time: 1min 8s


SVC(kernel='linear', probability=True)

In [8]:
pred_y = model.predict(X_test_2d.reshape(X_test_2d.shape[0],640*80*2))
precision_recall_fscore_support(y_test_2d, pred_y, average='weighted')

(1.0, 1.0, 1.0, None)

In [9]:
# %%time
scores = cross_val_score(model, total_images_2d.reshape(total_images_2d.shape[0],640*80*2), total_labels, cv=4)
print(scores)
print("%0.2f accuracy with a standard deviation of %0.2f" % (scores.mean(), scores.std()))

[1. 1. 1. 1.]
1.00 accuracy with a standard deviation of 0.00


### Generated Using Our 1d (d) Input

In [ ]:
original_d = np.load("assets/d_complete_data.npz", allow_pickle=True)['arr_0']
generated_d = np.load("generated_data/d_gen.npz", allow_pickle=True)['arr_0']
total_images_d = np.vstack((original_d, generated_d))
X_train_d, X_test_d, y_train_d, y_test_d = train_test_split(total_images_d, total_labels, test_size=0.33, random_state=42)

In [ ]:
%%time
model = SVC(kernel='linear', probability=True)
model.fit(X_train_d.reshape(X_train_d.shape[0],640*80), X_test_d)

In [ ]:
scores = cross_val_score(model, total_images_d.reshape(total_images_d.shape[0],640*80), total_labels, cv=4)
print(scores)
print("%0.2f accuracy with a standard deviation of %0.2f" % (scores.mean(), scores.std()))

## CNN model

In [1]:
tf.compat.v1.set_random_seed(2019)

In [2]:
def makeCNN(shape):
	model = tf.keras.models.Sequential([
		tf.keras.layers.Conv2D(16,(3,3),activation = "relu" , input_shape = shape) ,
		tf.keras.layers.MaxPooling2D(2,2),
		tf.keras.layers.Conv2D(32,(3,3),activation = "relu") ,
		tf.keras.layers.MaxPooling2D(2,2),
		tf.keras.layers.Conv2D(64,(3,3),activation = "relu") ,
		tf.keras.layers.MaxPooling2D(2,2),
		tf.keras.layers.Conv2D(128,(3,3),activation = "relu"),
		tf.keras.layers.MaxPooling2D(2,2),
		tf.keras.layers.Flatten(),
		tf.keras.layers.Dense(300,activation="relu"),      #Adding the Hidden layer
		tf.keras.layers.Dropout(0.1,seed = 2022),
		tf.keras.layers.Dense(50,activation ="relu"),
		tf.keras.layers.Dropout(0.2,seed = 2022),
		tf.keras.layers.Dense(1,activation = "sigmoid")   #Adding the Output Layer
	])
	print(model.summary())
	return model

In [3]:
epochs = 5

### Generated Using Daniel 3d (x,y,z) Input

In [35]:
cnn = makeCNN((80, 640, 3))
cnn.compile(optimizer=Adam(learning_rate=0.001),
			loss='binary_crossentropy',
			metrics=['acc', 'mse'])

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_16 (Conv2D)          (None, 78, 638, 16)       448       
                                                                 
 max_pooling2d_16 (MaxPoolin  (None, 39, 319, 16)      0         
 g2D)                                                            
                                                                 
 conv2d_17 (Conv2D)          (None, 37, 317, 32)       4640      
                                                                 
 max_pooling2d_17 (MaxPoolin  (None, 18, 158, 32)      0         
 g2D)                                                            
                                                                 
 conv2d_18 (Conv2D)          (None, 16, 156, 64)       18496     
                                                                 
 max_pooling2d_18 (MaxPoolin  (None, 8, 78, 64)       

In [37]:
# train the model
history = cnn.fit(X_train_3d_d, y_train_3d_d, epochs=epochs,
				  validation_data=(X_test_3d_d, y_test_3d_d))

Epoch 1/5
105/105 [==============================] - 49s 462ms/step - loss: 0.0538 - acc: 0.9764 - mse: 0.0162 - val_loss: 9.5193e-06 - val_acc: 1.0000 - val_mse: 4.9612e-10
Epoch 2/5
105/105 [==============================] - 43s 410ms/step - loss: 0.0017 - acc: 0.9991 - mse: 5.6535e-04 - val_loss: 3.9135e-04 - val_acc: 1.0000 - val_mse: 8.1608e-07
Epoch 3/5
105/105 [==============================] - 43s 408ms/step - loss: 0.0046 - acc: 0.9997 - mse: 4.1988e-04 - val_loss: 3.9134e-06 - val_acc: 1.0000 - val_mse: 8.4621e-11
Epoch 4/5
105/105 [==============================] - 59s 562ms/step - loss: 1.6117e-05 - acc: 1.0000 - mse: 7.9550e-08 - val_loss: 3.3673e-07 - val_acc: 1.0000 - val_mse: 6.4389e-13
Epoch 5/5
105/105 [==============================] - 53s 503ms/step - loss: 5.9063e-06 - acc: 1.0000 - mse: 6.4031e-09 - val_loss: 6.0012e-08 - val_acc: 1.0000 - val_mse: 2.0961e-14


## Generated using Our 3d (x,y,z) input

In [30]:
cnn = makeCNN((80, 640, 3))

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_12 (Conv2D)          (None, 78, 638, 16)       448       
                                                                 
 max_pooling2d_12 (MaxPoolin  (None, 39, 319, 16)      0         
 g2D)                                                            
                                                                 
 conv2d_13 (Conv2D)          (None, 37, 317, 32)       4640      
                                                                 
 max_pooling2d_13 (MaxPoolin  (None, 18, 158, 32)      0         
 g2D)                                                            
                                                                 
 conv2d_14 (Conv2D)          (None, 16, 156, 64)       18496     
                                                                 
 max_pooling2d_14 (MaxPoolin  (None, 8, 78, 64)       

In [33]:
cnn.compile(optimizer=Adam(learning_rate=0.001),
			loss='binary_crossentropy',
			metrics=['acc', 'mse'])

In [34]:
# train the model
history = cnn.fit(X_train_3d, y_train_3d, epochs=epochs,
				  validation_data=(X_test_3d, y_test_3d))

Epoch 1/5
105/105 [==============================] - 59s 548ms/step - loss: 0.0632 - accuracy: 0.9761 - acc: 0.9761 - mse: 0.0174 - val_loss: 4.7815e-06 - val_accuracy: 1.0000 - val_acc: 1.0000 - val_mse: 8.5754e-09
Epoch 2/5
105/105 [==============================] - 42s 405ms/step - loss: 1.8806e-04 - accuracy: 1.0000 - acc: 1.0000 - mse: 2.2014e-05 - val_loss: 3.5616e-07 - val_accuracy: 1.0000 - val_acc: 1.0000 - val_mse: 7.2484e-13
Epoch 3/5
105/105 [==============================] - 42s 403ms/step - loss: 6.1552e-06 - accuracy: 1.0000 - acc: 1.0000 - mse: 5.4640e-09 - val_loss: 9.5601e-08 - val_accuracy: 1.0000 - val_acc: 1.0000 - val_mse: 5.3549e-14
Epoch 4/5
105/105 [==============================] - 42s 402ms/step - loss: 5.0128e-06 - accuracy: 1.0000 - acc: 1.0000 - mse: 6.6771e-09 - val_loss: 4.0079e-08 - val_accuracy: 1.0000 - val_acc: 1.0000 - val_mse: 9.5278e-15
Epoch 5/5
105/105 [==============================] - 42s 403ms/step - loss: 6.3396e-06 - accuracy: 1.0000 - acc:

## Generated using 2d (d,z) as input

### Normal train test split

In [55]:
cnn = makeCNN((80,640,2))
cnn.compile(optimizer=Adam(learning_rate=0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_228 (Conv2D)         (None, 78, 638, 16)       304       
                                                                 
 max_pooling2d_48 (MaxPoolin  (None, 39, 319, 16)      0         
 g2D)                                                            
                                                                 
 conv2d_229 (Conv2D)         (None, 37, 317, 32)       4640      
                                                                 
 max_pooling2d_49 (MaxPoolin  (None, 18, 158, 32)      0         
 g2D)                                                            
                                                                 
 conv2d_230 (Conv2D)         (None, 16, 156, 64)       18496     
                                                                 
 max_pooling2d_50 (MaxPoolin  (None, 8, 78, 64)      

In [58]:
# train the model
history = cnn.fit(X_train_2d, y_train_2d, epochs=epochs,
                    validation_data=(X_test_2d, y_test_2d))

Epoch 1/5
105/105 [==============================] - 54s 499ms/step - loss: 0.0439 - accuracy: 0.9773 - val_loss: 1.1339e-06 - val_accuracy: 1.0000
Epoch 2/5
105/105 [==============================] - 48s 458ms/step - loss: 2.6593e-05 - accuracy: 1.0000 - val_loss: 1.1586e-07 - val_accuracy: 1.0000
Epoch 3/5
105/105 [==============================] - 52s 496ms/step - loss: 6.1218e-06 - accuracy: 1.0000 - val_loss: 9.8876e-09 - val_accuracy: 1.0000
Epoch 4/5
105/105 [==============================] - 56s 529ms/step - loss: 1.3453e-06 - accuracy: 1.0000 - val_loss: 2.8946e-09 - val_accuracy: 1.0000
Epoch 5/5
105/105 [==============================] - 47s 451ms/step - loss: 1.9519e-07 - accuracy: 1.0000 - val_loss: 1.8729e-09 - val_accuracy: 1.0000


### Generated Using Our 1d (d) Input

In [ ]:
cnn = makeCNN((80,640,1))
cnn.compile(optimizer=Adam(learning_rate=0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
# train the model
history = cnn.fit(X_train_d, y_train_d, epochs=epochs,
                    validation_data=(X_test_d, y_test_d))

# Metrics Tested on 3d Reconstructions